## Methodology Overview
 - Filter data to only contain candidates that won their respective election
 - Sum up repeat cells as well as tally votes so that each winning candidate has a single index
 - For each municipality-year index, see if person changed and add this to an additional column accordingly
     - -1: 2004 data (no prior data to compare to in given timeframe)
     -  0: Winning candidate was the same as the previous year
     -  1: new mayor

In [4]:
import pandas as pd
import numpy as np
import unidecode
from tqdm import tqdm

In [2]:
df = pd.read_csv('mayors.csv')
df.shape

C:\Users\kabir\miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(84255, 26)

In [3]:
# filter for only winning columns
elected_df = df[df["DESC_SIT_CAND_TOT"]=="ELEITO"]
print(elected_df.shape)
print(elected_df["DESC_SIT_CAND_TOT"].unique())

(25247, 26)
['ELEITO']


In [4]:
elected_df.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,...,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
2,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,45,27.0,...,DEFERIDO,1,ELEITO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,12.0,CONTINUANDO O PROGRESSO,#NE#,2173
3,ASSIS BRASIL,2004,1,ELEICOES 2004,AC,1570,1570,11,13,21.0,...,DEFERIDO,1,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,4.0,FRENTE POPULAR DE ASSIS BRASIL-FPA,#NE#,1675
5,BRASILEIA,2004,1,ELEICOES 2004,AC,1058,1058,11,13,66.0,...,DEFERIDO,1,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,11.0,FPB (FRENTE POPULAR DE BRASILÉIA),#NE#,5592
7,BUJARI,2004,1,ELEICOES 2004,AC,1007,1007,11,13,39.0,...,DEFERIDO,1,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,14.0,FRENTE POPULAR DE BUJARI,#NE#,1794
10,CAPIXABA,2004,1,ELEICOES 2004,AC,1015,1015,11,14,75.0,...,DEFERIDO,1,ELEITO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,27.0,FRENTE SOCIALISTA TRABALHISTA CRISTÃ,#NE#,1155


In [8]:
def normalize_str_arr(arr):
    norm_arr = []
    for item in arr:
        norm_arr.append(unidecode.unidecode(item.lower()))
    return norm_arr

In [6]:
norm_muni_arr = normalize_str_arr(elected_df["NOME_MUNICIPIO"].tolist())
elected_df["NORM_NOME_MUNICIPIO"] = norm_muni_arr

C:\Users\kabir\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
munis = pd.read_csv('41_Municipality_GG2.10_condensed._dupes_removed.csv')
munis_arr = munis["ADM2_NAME"].tolist()

In [8]:
filter_municipalities = normalize_str_arr(munis_arr)
elected_df = elected_df[elected_df["NORM_NOME_MUNICIPIO"].isin(filter_municipalities)]
elected_df.shape

(1634, 27)

In [9]:
elected_df["NORM_NOME_MUNICIPIO"].value_counts()

rio de janeiro           388
nova iguacu               36
cuiaba                    29
porto velho               28
campinas                  26
                        ... 
alto paraiso de goias      3
viseu                      3
santana do livramento      3
senador la rocque          3
parati                     2
Name: NORM_NOME_MUNICIPIO, Length: 238, dtype: int64

In [10]:
def generate_id_col(df):
    df["ID"] = df["ANO_ELEICAO"].astype(str).str.cat(df[['NOME_MUNICIPIO', 'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO']],
                                                    sep='-')
    return df

In [11]:
id_df = generate_id_col(elected_df)
id_df.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,...,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID
3,ASSIS BRASIL,2004,1,ELEICOES 2004,AC,1570,1570,11,13,21.0,...,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,4.0,FRENTE POPULAR DE ASSIS BRASIL-FPA,#NE#,1675,assis brasil,2004-ASSIS BRASIL-MANOEL BATISTA DE ARAÚJO-MANOEL
17,EPITACIOLANDIA,2004,1,ELEICOES 2004,AC,1112,1112,11,40,160.0,...,ELEITO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,32.0,FRENTE POPULAR DE EPITACIOLÂNDIA - FPE,#NE#,2844,epitaciolandia,2004-EPITACIOLANDIA-JOSE RONALDO PESSOA PEREIR...
19,FEIJO,2004,1,ELEICOES 2004,AC,1139,1139,11,13,34.0,...,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,5.0,#NULO#,#NE#,5337,feijo,2004-FEIJO-FRANCIMAR FERNANDES DE ALBUQUERQUE-...
22,JORDAO,2004,1,ELEICOES 2004,AC,1104,1104,11,13,1.0,...,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,4.0,FRENTE POPULAR DE JORDÃO,#NE#,1345,jordao,2004-JORDAO-HILÁRIO DE HOLANDA MELO-MELO
25,MANCIO LIMA,2004,1,ELEICOES 2004,AC,1090,1090,11,22,88.0,...,ELEITO,22,PL,PARTIDO LIBERAL,12.0,ALIANÇA DEMOCRÁTICA LIBERAL,#NE#,2327,mancio lima,2004-MANCIO LIMA-LUIZ HELOSMAN DE FIGUEIREDO-L...


In [12]:
COLUMNS = id_df.columns.tolist()

In [13]:
id_df["ID"].value_counts()

2004-RIO DE JANEIRO-CESAR EPITÁCIO MAIA-CESAR MAIA                     97
2016-RIO DE JANEIRO-MARCELO BEZERRA CRIVELLA-CRIVELLA                  97
2012-RIO DE JANEIRO-EDUARDO DA COSTA PAES-EDUARDO PAES                 97
2008-RIO DE JANEIRO-EDUARDO DA COSTA PAES-EDUARDO PAES                 97
2004-CUIABA-WILSON PEREIRA DOS SANTOS-WILSON SANTOS                    11
                                                                       ..
2008-SANTA ISABEL DO RIO NEGRO-ELIETE DA CUNHA BELEZA-ELIETE BELEZA     1
2004-OEIRAS DO PARA-DULCÍDIO FERREIRA PINHEIRO-DULCÍDIO                 1
2004-GUATAMBU-LAURI JORGE GERELLI-LAURI JORGE GERELLI                   1
2008-MANICORÉ-EMERSON PEDRACA DE FRANCA-EMERSON                         1
2012-MAMBAÍ-JAVAN LOPES DA SILVA JUNIOR-JUNIOR                          1
Name: ID, Length: 1046, dtype: int64

In [14]:
id_df[id_df["ID"]=="2016-SÃO PAULO-JOÃO AGRIPINO DA COSTA DORIA JUNIOR-JOÃO DORIA"]

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,...,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID


In [15]:
id_df_2004 = id_df[id_df["ID"]==2004]
id_df_2004

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,...,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID


In [16]:
id_df_2004 = id_df.where(id_df['ANO_ELEICAO'] == 2004)
id_df_2004 = id_df_2004.dropna(how='all')

id_df_2008 = id_df.where(id_df['ANO_ELEICAO'] == 2008)
id_df_2008 = id_df_2008.dropna(how='all')

id_df_2012 = id_df.where(id_df['ANO_ELEICAO'] == 2012)
id_df_2012 = id_df_2012.dropna(how='all')

id_df_2016 = id_df.where(id_df['ANO_ELEICAO'] == 2016)
id_df_2016 = id_df_2016.dropna(how='all')

In [17]:
id_df_2004["ID"].value_counts()

2004-RIO DE JANEIRO-CESAR EPITÁCIO MAIA-CESAR MAIA              97
2004-CUIABA-WILSON PEREIRA DOS SANTOS-WILSON SANTOS             11
2004-NOVA IGUACU-LUIZ LINDBERGH FARIAS FILHO-LINDBERG FARIAS     9
2004-PORTO VELHO-ROBERTO EDUARDO SOBRINHO-ROBERTO SOBRINHO       7
2004-CAMPINAS-HÉLIO DE OLIVEIRA SANTOS-DR. HÉLIO                 6
                                                                ..
2004-UBATUBA-EDUARDO DE SOUZA CESAR-EDUARDO CESAR                1
2004-ITANHAEM-JOÃO CARLOS FORSSELL NETO-FORSSELL                 1
2004-MANICORE-EMERSON PEDRAÇA DE FRANÇA-EMERSON                  1
2004-TAMANDARE-PAULO ROMERO PEREIRA DA SILVA-PAULINHO            1
2004-ARRAIAL DO CABO-HENRIQUE SERGIO MELMAN-HENRIQUE MELMAN      1
Name: ID, Length: 254, dtype: int64

In [18]:
def clean_df(df):
    """
    Combine duplicate candidate entries by summing up their votes and collapsing the remaining
    info into a single row. So, each municipality-year-candidatename should be almost always unique.
    1) Group by ID
    2) Sum groupby and save ID and TOTAL VOTOS (which are now combined upon summation) in dict - {ID: total votes}
    3) Store the first row with this ID in numpy array
    4) Delete all cells containing this ID
    5) Change the total votes value in the numpy array with the summed value
    6) Re-add in this array to the dataframe
    7) Repeat for all IDs that have value counts >1
    8) Re-sort dataframe alphabetically by NOME MUNICIPIO to match original convention and restore order to dataframe
    """
    gb = df.groupby(["ID"], as_index=False).sum()
    
    dict_yr = dict(zip(gb.ID, gb.TOTAL_VOTOS))
    
    unique_rows = []
    for key in tqdm(dict_yr.keys()):
        subset_df = df[df["ID"]==key]
        unique_rows.append(subset_df.iloc[0].to_numpy())
    
    for key in tqdm(dict_yr.keys()):
        for r in unique_rows:
            if key in r:
                r[-3] = dict_yr[key]
                
    clean_df = pd.DataFrame(unique_rows, columns=COLUMNS)
    clean_df.sort_values('NOME_MUNICIPIO')
    
    return clean_df

In [19]:
clean_df_2004 = clean_df(id_df_2004)
clean_df_2004

100%|███████████████████████████████████████████████████████████████████████████████| 254/254 [00:02<00:00, 115.99it/s]


,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,...,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID
0,ABELARDO LUZ,2004.0,1.0,ELEICOES 2004,SC,80012.0,80012.0,11.0,15.0,66.0,...,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,13.0,POR UM ABELARDO MELHOR,#NE#,4945.0,abelardo luz,2004-ABELARDO LUZ-NERCI SANTIN-SANTIN
1,AIUABA,2004.0,1.0,ELEICOES 2004,CE,13072.0,13072.0,11.0,45.0,1.0,...,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1.0,#NULO#,#NE#,5463.0,aiuaba,2004-AIUABA-RAMILSON ARAUJO MORAES-RAMILSON
2,ALMEIRIM,2004.0,1.0,ELEICOES 2004,PA,4090.0,4090.0,11.0,45.0,52.0,...,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,19.0,UNIÃO POR ALMEIRIM E MONTE DOURADO (PSDB/PFL/P...,#NE#,6131.0,almeirim,2004-ALMEIRIM-GANDOR CALIL HAGE NETO-GANDOR HAGE
3,ALTAMIRA,2004.0,1.0,ELEICOES 2004,PA,4111.0,4111.0,11.0,45.0,221.0,...,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,49.0,FRENTE PELA LIBERTAÇÃO DE ALTAMIRA,#NE#,17547.0,altamira,2004-ALTAMIRA-ODILEIDA MARIA DE SOUSA SAMPAIO-...
4,ALTO ALEGRE,2004.0,1.0,ELEICOES 2004,RS,84808.0,84808.0,11.0,15.0,1.0,...,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,2.0,UNIDOS POR ALTO ALEGRE,#NE#,748.0,alto alegre,2004-ALTO ALEGRE-ETELVINO MORGAN-VINO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,UIRAMUTA,2004.0,1.0,ELEICOES 2004,RR,3107.0,3107.0,11.0,13.0,351.0,...,ELEITO,13.0,PT,PARTIDO DOS TRABALHADORES,136.0,UIRAMUTÃ PARA TODOS,#NE#,968.0,uiramuta,2004-UIRAMUTA-FLORANY MARIA DOS SANTOS MOTA-FLORA
250,UNA,2004.0,1.0,ELEICOES 2004,BA,39497.0,39497.0,11.0,70.0,155.0,...,ELEITO,70.0,PT do B,PARTIDO TRABALHISTA DO BRASIL,25.0,UNA PARA TODOS,#NE#,8155.0,una,2004-UNA-JOSÉ BISPO SANTOS-ZÉ PRETINHO
251,VITOR MEIRELES,2004.0,1.0,ELEICOES 2004,SC,80128.0,80128.0,11.0,15.0,26.0,...,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,7.0,VITOR MEIRELES NO CAMINHO CERTO,#NE#,1515.0,vitor meireles,2004-VITOR MEIRELES-LOURIVAL LUNELLI-LOURIVAL
252,VITORIA DO JARI,2004.0,1.0,ELEICOES 2004,AP,6122.0,6122.0,11.0,70.0,58.0,...,ELEITO,70.0,PT do B,PARTIDO TRABALHISTA DO BRASIL,14.0,'VITÓRIA PARA TODOS',#NE#,3453.0,vitoria do jari,2004-VITORIA DO JARI-ADELSON FERREIRA DE FIGUE...


In [20]:
clean_df_2008 = clean_df(id_df_2008)
clean_df_2008.shape

100%|███████████████████████████████████████████████████████████████████████████████| 268/268 [00:02<00:00, 111.40it/s]


(268, 28)

In [21]:
clean_df_2012 = clean_df(id_df_2012)
clean_df_2012.shape

100%|███████████████████████████████████████████████████████████████████████████████| 261/261 [00:02<00:00, 113.73it/s]


(261, 28)

In [22]:
clean_df_2016 = clean_df(id_df_2016)
clean_df_2016.shape

100%|███████████████████████████████████████████████████████████████████████████████| 263/263 [00:02<00:00, 113.38it/s]


(263, 28)

In [23]:
mayoral_dfs = [clean_df_2004, clean_df_2008, clean_df_2012, clean_df_2016]

In [24]:
result = pd.concat(mayoral_dfs)
result.to_csv("elected_mayors_gee_filtered.csv", encoding='utf-8-sig', index=False)

## Part 2

In [29]:
mayor_csv = pd.read_csv("elected_mayors_gee_filtered.csv")
mayor_csv.shape

(1046, 28)

In [30]:
mayor_csv.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID
0,ABELARDO LUZ,2004.0,1.0,ELEICOES 2004,SC,80012.0,80012.0,11.0,15.0,66.0,NERCI SANTIN,SANTIN,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,13.0,POR UM ABELARDO MELHOR,#NE#,4945.0,abelardo luz,2004-ABELARDO LUZ-NERCI SANTIN-SANTIN
1,AIUABA,2004.0,1.0,ELEICOES 2004,CE,13072.0,13072.0,11.0,45.0,1.0,RAMILSON ARAUJO MORAES,RAMILSON,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1.0,#NULO#,#NE#,5463.0,aiuaba,2004-AIUABA-RAMILSON ARAUJO MORAES-RAMILSON
2,ALMEIRIM,2004.0,1.0,ELEICOES 2004,PA,4090.0,4090.0,11.0,45.0,52.0,GANDOR CALIL HAGE NETO,GANDOR HAGE,PREFEITO,-1.0,#NE#,4,SUB JUDICE,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,19.0,UNIÃO POR ALMEIRIM E MONTE DOURADO (PSDB/PFL/P...,#NE#,6131.0,almeirim,2004-ALMEIRIM-GANDOR CALIL HAGE NETO-GANDOR HAGE
3,ALTAMIRA,2004.0,1.0,ELEICOES 2004,PA,4111.0,4111.0,11.0,45.0,221.0,ODILEIDA MARIA DE SOUSA SAMPAIO,ODILEIDA,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,49.0,FRENTE PELA LIBERTAÇÃO DE ALTAMIRA,#NE#,17547.0,altamira,2004-ALTAMIRA-ODILEIDA MARIA DE SOUSA SAMPAIO-...
4,ALTO ALEGRE,2004.0,1.0,ELEICOES 2004,RS,84808.0,84808.0,11.0,15.0,1.0,ETELVINO MORGAN,VINO,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,2.0,UNIDOS POR ALTO ALEGRE,#NE#,748.0,alto alegre,2004-ALTO ALEGRE-ETELVINO MORGAN-VINO


In [31]:
ids = mayor_csv["ID"].tolist()
normalized_ids = normalize_str_arr(arr=ids)
mayor_csv["NORM_ID"] = normalized_ids

In [32]:
pd.set_option('display.max_columns', None)

In [33]:
mayor_csv[mayor_csv["NORM_ID"].str.contains("humaita")]

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID,NORM_ID
96,HUMAITA,2004.0,1.0,ELEICOES 2004,RS,86959.0,86959.0,11.0,14.0,1.000000e+00,LUIZ NELSON SCHMATZ,LUIZ NELSON SCHMATZ (BUGIO),PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,14.0,PTB,PARTIDO TRABALHISTA BRASILEIRO,2.000000e+00,FRENTE POPULAR,#NE#,1962.0,humaita,2004-HUMAITA-LUIZ NELSON SCHMATZ-LUIZ NELSON S...,2004-humaita-luiz nelson schmatz-luiz nelson s...
97,HUMAITA,2004.0,1.0,ELEICOES 2004,AM,2356.0,2356.0,11.0,14.0,6.300000e+01,ROBERTO RUI GUERRA DE SOUZA,ROBERTO RUI,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,14.0,PTB,PARTIDO TRABALHISTA BRASILEIRO,3.200000e+01,COLIGAÇÃO PELA MUDANÇA URGENTE,#NE#,8703.0,humaita,2004-HUMAITA-ROBERTO RUI GUERRA DE SOUZA-ROBER...,2004-humaita-roberto rui guerra de souza-rober...
356,HUMAITÁ,2008.0,1.0,ELEIÇÕES 2008,RS,86959.0,86959.0,11.0,11.0,2.159700e+04,CESAR SCHWADE,CESAR SCHWADE,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,11.0,PP,PARTIDO PROGRESSISTA,2.302000e+03,FRENTE POPULAR,#NE#,2215.0,humaita,2008-HUMAITÁ-CESAR SCHWADE-CESAR SCHWADE,2008-humaita-cesar schwade-cesar schwade
357,HUMAITÁ,2008.0,1.0,ELEIÇÕES 2008,AM,2356.0,2356.0,11.0,15.0,8.940000e+02,JOSÉ CIDENEI LOBO DO NASCIMENTO,DEDEI LOBO,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,1.430000e+02,CORAGEM PARA MUDAR I,#NE#,9962.0,humaita,2008-HUMAITÁ-JOSÉ CIDENEI LOBO DO NASCIMENTO-D...,2008-humaita-jose cidenei lobo do nascimento-d...
621,HUMAITÁ,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,AM,2356.0,2356.0,11.0,15.0,4.000000e+10,JOSÉ CIDENEI LOBO DO NASCIMENTO,DEDEI LOBO,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,4.000000e+10,AVANÇA HUMAITÁ,PDT / PMDB / PTN / PSC / PPS / DEM / PMN / PSB...,11093.0,humaita,2012-HUMAITÁ-JOSÉ CIDENEI LOBO DO NASCIMENTO-D...,2012-humaita-jose cidenei lobo do nascimento-d...
622,HUMAITÁ,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,RS,86959.0,86959.0,11.0,11.0,2.100000e+11,LUIZ CARLOS SANDRI,LUIZ CARLOS,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,11.0,PP,PARTIDO PROGRESSISTA,2.100000e+11,FRENTE POPULAR,PP / PTB / PSDB,2320.0,humaita,2012-HUMAITÁ-LUIZ CARLOS SANDRI-LUIZ CARLOS,2012-humaita-luiz carlos sandri-luiz carlos
883,HUMAITÁ,2016.0,1.0,ELEIÇÕES MUNICIPAIS 2016,RS,86959.0,86959.0,11.0,11.0,2.100000e+11,FERNANDO WEGMANN,FERNANDO WEGMANN,PREFEITO,2.0,DEFERIDO,12,APTO,1.0,ELEITO,11.0,PP,Partido Progressista,2.100000e+11,FRENTE POPULAR,PP / PTB / PSDB,1933.0,humaita,2016-HUMAITÁ-FERNANDO WEGMANN-FERNANDO WEGMANN,2016-humaita-fernando wegmann-fernando wegmann
884,HUMAITÁ,2016.0,1.0,ELEIÇÕES MUNICIPAIS 2016,AM,2356.0,2356.0,11.0,90.0,4.000000e+10,HERIVANEO VIEIRA DE OLIVEIRA,HERIVANEO SEIXAS,PREFEITO,2.0,DEFERIDO,12,APTO,1.0,ELEITO,90.0,PROS,Partido Republicano da Ordem Social,4.000000e+10,HUMAITÁ DE TODOS NÓS,PROS / PSD / PTN / PSC / DEM / PMN / PV / PRB ...,10752.0,humaita,2016-HUMAITÁ-HERIVANEO VIEIRA DE OLIVEIRA-HERI...,2016-humaita-herivaneo vieira de oliveira-heri...


In [34]:
mayor_csv["TURNOVER_STATUS"]=0

In [35]:
mayor_csv.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID,NORM_ID,TURNOVER_STATUS
0,ABELARDO LUZ,2004.0,1.0,ELEICOES 2004,SC,80012.0,80012.0,11.0,15.0,66.0,NERCI SANTIN,SANTIN,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,13.0,POR UM ABELARDO MELHOR,#NE#,4945.0,abelardo luz,2004-ABELARDO LUZ-NERCI SANTIN-SANTIN,2004-abelardo luz-nerci santin-santin,0
1,AIUABA,2004.0,1.0,ELEICOES 2004,CE,13072.0,13072.0,11.0,45.0,1.0,RAMILSON ARAUJO MORAES,RAMILSON,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1.0,#NULO#,#NE#,5463.0,aiuaba,2004-AIUABA-RAMILSON ARAUJO MORAES-RAMILSON,2004-aiuaba-ramilson araujo moraes-ramilson,0
2,ALMEIRIM,2004.0,1.0,ELEICOES 2004,PA,4090.0,4090.0,11.0,45.0,52.0,GANDOR CALIL HAGE NETO,GANDOR HAGE,PREFEITO,-1.0,#NE#,4,SUB JUDICE,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,19.0,UNIÃO POR ALMEIRIM E MONTE DOURADO (PSDB/PFL/P...,#NE#,6131.0,almeirim,2004-ALMEIRIM-GANDOR CALIL HAGE NETO-GANDOR HAGE,2004-almeirim-gandor calil hage neto-gandor hage,0
3,ALTAMIRA,2004.0,1.0,ELEICOES 2004,PA,4111.0,4111.0,11.0,45.0,221.0,ODILEIDA MARIA DE SOUSA SAMPAIO,ODILEIDA,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,49.0,FRENTE PELA LIBERTAÇÃO DE ALTAMIRA,#NE#,17547.0,altamira,2004-ALTAMIRA-ODILEIDA MARIA DE SOUSA SAMPAIO-...,2004-altamira-odileida maria de sousa sampaio-...,0
4,ALTO ALEGRE,2004.0,1.0,ELEICOES 2004,RS,84808.0,84808.0,11.0,15.0,1.0,ETELVINO MORGAN,VINO,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,2.0,UNIDOS POR ALTO ALEGRE,#NE#,748.0,alto alegre,2004-ALTO ALEGRE-ETELVINO MORGAN-VINO,2004-alto alegre-etelvino morgan-vino,0


In [36]:
check_arr = []

In [37]:
visited = []
mayor_change = [0]*mayor_csv.shape[0]

for idx, row in tqdm(mayor_csv.iterrows(), total=mayor_csv.shape[0]):
    # grab ID without year
    cur_id = row[-2][5:]
    
    # if ID is not one that we have already processed, then move forward
    if idx not in visited:
        # grab all indexes where repeating normalized IDs are found
        idxs = mayor_csv[mayor_csv["NORM_ID"].str.contains(cur_id, regex = False)].index.tolist()
        idx_yrs = []
        
        for i in range(len(idxs)):
            idx_yr = mayor_csv.loc[idxs[i], 'ANO_ELEICAO']
            idx_yrs.append(idx_yr)
            # turnover checking
            if idx_yr  == 2004:
                mayor_csv.loc[idxs[i], 'TURNOVER_STATUS']=-1
            elif (i==0) and (idx_yr != 2004):
                mayor_csv.loc[idxs[i], 'TURNOVER_STATUS']=1
            else:
                mayor_csv.loc[idxs[i], 'TURNOVER_STATUS']=0
            visited.append(idxs[i])
        
        for i in range(len(idx_yrs)):
            if idx_yrs[i]==2004:
                mayor_change[idxs[i]]=-1
            elif (i==0) and (idx_yrs[i]!=2004):
                mayor_change[idxs[i]]=1
            elif i!=0:
                if (idx_yrs[i]-4)!=idx_yrs[i-1]:
                    check_arr.append(idxs[i])
                    mayor_change[idxs[i]]=1
                else:
                    mayor_change[idxs[i]]=0

100%|████████████████████████████████████████████████████████████████████████████| 1046/1046 [00:00<00:00, 1321.87it/s]


In [44]:
mayor_csv["MAYOR_CHANGE"] = np.array(mayor_change)

In [38]:
len(mayor_change)

1046

In [39]:
mayor_change

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [41]:
check_arr

[784,
 532,
 796,
 808,
 812,
 821,
 561,
 570,
 850,
 580,
 845,
 849,
 852,
 858,
 607,
 630,
 632,
 900,
 911,
 650,
 916,
 671,
 953,
 692,
 696,
 960,
 710,
 981,
 1012,
 753,
 763,
 780,
 800,
 841,
 909,
 926,
 929,
 942,
 963,
 973,
 981,
 988,
 992,
 1045,
 799]

In [46]:
mayor_csv.loc[607]

NOME_MUNICIPIO                                              FLORESTA
ANO_ELEICAO                                                     2012
NUM_TURNO                                                          1
DESCRICAO_ELEICAO                             ELEIÇÃO MUNICIPAL 2012
SIGLA_UF                                                          PR
SIGLA_UE                                                       75558
CODIGO_MUNICIPIO                                               75558
CODIGO_CARGO                                                      11
NUMERO_CAND                                                       11
SQ_CANDIDATO                                                 1.6e+11
NOME_CANDIDATO                                     JOSE ROBERTO RUIZ
NOME_URNA_CANDIDATO                                       ZÉ ROBERTO
DESCRICAO_CARGO                                             PREFEITO
COD_SIT_CAND_SUPERIOR                                             12
DESC_SIT_CAND_SUPERIOR            

In [47]:
mayor_csv[mayor_csv["NORM_ID"].str.contains("floresta")]

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID,NORM_ID,TURNOVER_STATUS,MAYOR_CHANGE
83,FLORESTA,2004.0,1.0,ELEICOES 2004,PE,24139.0,24139.0,11.0,45.0,1.000000e+00,AFONSO AUGUSTO FERRAZ,AFONSO AUGUSTO FERRAZ,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,45.0,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,2.000000e+00,DO JEITO QUE O POVO QUER,#NE#,8109.0,floresta,2004-FLORESTA-AFONSO AUGUSTO FERRAZ-AFONSO AUG...,2004-floresta-afonso augusto ferraz-afonso aug...,-1,-1
84,FLORESTA,2004.0,1.0,ELEICOES 2004,PR,75558.0,75558.0,11.0,11.0,2.350000e+02,JOSÉ ROBERTO RUIZ,ZÉ ROBERTO,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,11.0,PP,PARTIDO PROGRESSISTA,4.700000e+01,O TRABALHO CONTINUA,#NE#,2074.0,floresta,2004-FLORESTA-JOSÉ ROBERTO RUIZ-ZÉ ROBERTO,2004-floresta-jose roberto ruiz-ze roberto,-1,-1
150,NISIA FLORESTA,2004.0,1.0,ELEICOES 2004,RN,17639.0,17639.0,11.0,25.0,4.600000e+01,GEORGE NEI FERREIRA,GEORGE,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,25.0,PFL,PARTIDO DA FRENTE LIBERAL,1.400000e+01,O POVO FAZ A VITÓRIA,#NE#,5683.0,nisia floresta,2004-NISIA FLORESTA-GEORGE NEI FERREIRA-GEORGE,2004-nisia floresta-george nei ferreira-george,-1,-1
342,FLORESTA,2008.0,1.0,ELEIÇÕES 2008,PR,75558.0,75558.0,11.0,15.0,8.952000e+03,ANTONIO FUENTES MARTINS,FUENTES,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,9.470000e+02,UNIÃO E TRABALHO,#NE#,2120.0,floresta,2008-FLORESTA-ANTONIO FUENTES MARTINS-FUENTES,2008-floresta-antonio fuentes martins-fuentes,1,1
343,FLORESTA,2008.0,1.0,ELEIÇÕES 2008,PE,24139.0,24139.0,11.0,40.0,2.702000e+03,ROSANGELA DE MOURA MANIÇOBA NOVAES FERRAZ,RORRÓ,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,40.0,PSB,PARTIDO SOCIALISTA BRASILEIRO,2.750000e+02,FLORESTA PARA TODOS,#NE#,8060.0,floresta,2008-FLORESTA-ROSANGELA DE MOURA MANIÇOBA NOVA...,2008-floresta-rosangela de moura manicoba nova...,1,1
417,NÍSIA FLORESTA,2008.0,1.0,ELEIÇÕES 2008,RN,17639.0,17639.0,11.0,25.0,8.020000e+02,GEORGE NEY FERREIRA,GEORGE,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,25.0,DEM,DEMOCRATAS,1.330000e+02,O POVO FAZ A VITORIA,#NE#,6982.0,nisia floresta,2008-NÍSIA FLORESTA-GEORGE NEY FERREIRA-GEORGE,2008-nisia floresta-george ney ferreira-george,1,1
607,FLORESTA,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,PR,75558.0,75558.0,11.0,11.0,1.600000e+11,JOSE ROBERTO RUIZ,ZÉ ROBERTO,PREFEITO,12.0,APTO,16,DEFERIDO COM RECURSO,1.0,ELEITO,11.0,PP,PARTIDO PROGRESSISTA,1.600000e+11,UNIÃO POR AMOR E RESPEITO AO NOSSO POVO,PP / PSDB,2261.0,floresta,2012-FLORESTA-JOSE ROBERTO RUIZ-ZÉ ROBERTO,2012-floresta-jose roberto ruiz-ze roberto,0,1
608,FLORESTA,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,PE,24139.0,24139.0,11.0,40.0,1.700000e+11,ROSANGELA DE MOURA MANIÇOBA NOVAES FERRAZ,RORRÓ,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,40.0,PSB,PARTIDO SOCIALISTA BRASILEIRO,1.700000e+11,FLORESTA NO RUMO CERTO,PSB / PT / PSD / PV / PSDC / PTN / PHS / PPL /...,9051.0,floresta,2012-FLORESTA-ROSANGELA DE MOURA MANIÇOBA NOVA...,2012-floresta-rosangela de moura manicoba nova...,0,0
682,NÍSIA FLORESTA,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,RN,17639.0,17639.0,11.0,25.0,2.000000e+11,CAMILA MACIEL FERREIRA,CAMILA,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,25.0,DEM,DEMOCRATAS,2.000000e+11,A UNIÃO PARA O POVO,PP / PMDB / PSC / PR / PPS / DEM / PSDB / PSD,8244.0,nisia floresta,2012-NÍSIA FLORESTA-CAMILA MACIEL FERREIRA-CAMILA,2012-nisia floresta-camila maciel ferreira-camila,1,1
868,FLORESTA,2016.0,1.0,ELEIÇÕES MUNICIPAIS 2016,PR,75558.0,75558.0,11.0,55.0,1.600000e+11,ADEMIR LUIZ MACIEL,DÊ,PREFEITO,2.0,DEFERIDO,12,APTO,1.0,ELEITO,55.0,PSD,Partido Social Democrático,1.600000e+11,DIAS MELHORES,PDT / PT / PTB / PMDB /

In [22]:
mayor_csv[mayor_csv["ANO_ELEICAO"]==2004]["TURNOVER_STATUS"].value_counts()

-1    254
Name: TURNOVER_STATUS, dtype: int64

In [23]:
574+254+217

1045

In [48]:
mayor_csv["TURNOVER_STATUS"].value_counts()

 1    575
-1    254
 0    217
Name: TURNOVER_STATUS, dtype: int64

In [49]:
mayor_csv["MAYOR_CHANGE"].value_counts()

 1    619
-1    254
 0    173
Name: MAYOR_CHANGE, dtype: int64

In [25]:
mayor_csv[mayor_csv["NOME_MUNICIPIO"]=="ABELARDO LUZ"]

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS,NORM_NOME_MUNICIPIO,ID,NORM_ID,TURNOVER_STATUS,MAYOR_CHANGE
0,ABELARDO LUZ,2004.0,1.0,ELEICOES 2004,SC,80012.0,80012.0,11.0,15.0,6.600000e+01,NERCI SANTIN,SANTIN,PREFEITO,-1.0,#NE#,2,DEFERIDO,1.0,ELEITO,15.0,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,1.300000e+01,POR UM ABELARDO MELHOR,#NE#,4945.0,abelardo luz,2004-ABELARDO LUZ-NERCI SANTIN-SANTIN,2004-abelardo luz-nerci santin-santin,-1,-1
254,ABELARDO LUZ,2008.0,1.0,ELEIÇÕES 2008,SC,80012.0,80012.0,11.0,13.0,3.337000e+03,DILMAR ANTONIO FANTINELLI,DILMAR,PREFEITO,12.0,APTO,2,DEFERIDO,1.0,ELEITO,13.0,PT,PARTIDO DOS TRABALHADORES,3.920000e+02,ABELARDO LUZ COM A FORÇA DO POVO,#NE#,5033.0,abelardo luz,2008-ABELARDO LUZ-DILMAR ANTONIO FANTINELLI-DI...,2008-abelardo luz-dilmar antonio fantinelli-di...,1,1
522,ABELARDO LUZ,2012.0,1.0,ELEIÇÃO MUNICIPAL 2012,SC,80012.0,80012.0,11.0,13.0,2.400000e+11,DILMAR ANTONIO FANTINELLI,DILMAR,PREFEITO,12.0,APTO,16,DEFERIDO COM RECURSO,1.0,ELEITO,13.0,PT,PARTIDO DOS TRABALHADORES,2.400000e+11,JUNTOS COM O POVO FAREMOS MAIS,PP / PDT / PT / PR / PSB / PC do B,5178.0,abelardo luz,2012-ABELARDO LUZ-DILMAR ANTONIO FANTINELLI-DI...,2012-abelardo luz-dilmar antonio fantinelli-di...,0,0
783,ABELARDO LUZ,2016.0,1.0,NOVA ELEIÇÃO SET/2017 - ABELARDO LUZ,SC,80012.0,80012.0,11.0,45.0,2.400000e+11,WILAMIR DOMINGOS CAVASSINI,CAVASSINI,PREFEITO,2.0,DEFERIDO,12,APTO,1.0,ELEITO,45.0,PSDB,Partido da Social Democracia Brasileira,2.400000e+11,ACELERA ABELARDO LUZ,PSDB / PP / PRP / PSD,4000.0,abelardo luz,2016-ABELARDO LUZ-WILAMIR DOMINGOS CAVASSINI-C...,2016-abelardo luz-wilamir domingos cavassini-c...,1,1


In [50]:
mayor_csv.to_csv("mayor_data_with_turnover_and_continuity_gee_filtered.csv", encoding='utf-8-sig', index=False)